# Densenet

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import keras as K
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
# from tensorflow.keras import datasets, layers, models

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import (Input, Conv2D,BatchNormalization, Concatenate,
                                    Activation, Add, MaxPooling2D, Dense, AveragePooling2D,
                                    GlobalAveragePooling2D)

2024-11-08 23:11:33.711495: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-08 23:11:33.719443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731125493.727979    3385 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731125493.730617    3385 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 23:11:33.739740: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Preparation for trainning - Train Split

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'Dataset of Breast mammography images with Masses/Image After Data Augmentation, Image Size 224x224/',
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(224, 224),
    batch_size=32,
)

Found 7632 files belonging to 8 classes.
Using 6106 files for training.


I0000 00:00:1731125497.422238    3385 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1756 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [4]:
classes_ = train_ds.class_names
len(classes_)

8

In [5]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'Dataset of Breast mammography images with Masses/Image After Data Augmentation, Image Size 224x224/',
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(224, 224),
    batch_size=32,
)

Found 7632 files belonging to 8 classes.
Using 1526 files for validation.


In [6]:
def normalize(image, label):
  return tf.cast(image/255., tf.float32), label

train_ds = train_ds.map(normalize)
val_ds = val_ds.map(normalize)

train_ds, val_ds

(<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>)

# Preparation for trainning - Test Split

In [7]:
train_ds_test = tf.keras.preprocessing.image_dataset_from_directory(
    'Dataset of Breast mammography images with Masses/Image Before Data Augmentation, Image Size 224x224/',
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(224, 224),
    batch_size=32,
)

Found 212 files belonging to 8 classes.
Using 170 files for training.


In [8]:
val_ds_test = tf.keras.preprocessing.image_dataset_from_directory(
    'Dataset of Breast mammography images with Masses/Image Before Data Augmentation, Image Size 224x224/',
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(224, 224),
    batch_size=32,
)

Found 212 files belonging to 8 classes.
Using 42 files for validation.


In [9]:
def normalize(image, label):
  return tf.cast(image/255., tf.float32), label

train_ds_test = train_ds_test.map(normalize)
val_ds_test = val_ds_test.map(normalize)

train_ds_test, val_ds_test

(<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>)

# Training

In [10]:
def DenseBlock(x, filters, growth_rate):
    inputs = x

    for i in range(growth_rate):
        x1 = BatchNormalization()(x)
        x1 = Activation('relu')(x1)
        x1 = Conv2D(4*growth_rate, kernel_size=1, use_bias=False)(x1)
        x1 = BatchNormalization()(x1)
        x1 = Activation('relu')(x1)
        x1 = Conv2D(growth_rate, kernel_size=3, padding='same', use_bias=False)(x1)
        x = Concatenate()([x, x1])

    return x

def TransitionLayer(x, reduction):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(int(x.shape[-1] * reduction), kernel_size=1, use_bias=False)(x)
    x = AveragePooling2D(pool_size=2, strides=2)(x)
    return x

def DenseNet(input_shape, depth, growth_rate, reduction, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(2*growth_rate, kernel_size=3, padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2, strides=2)(x)

    for i in range(int(depth/growth_rate)-1):
        x = DenseBlock(x, filters=x.shape[-1], growth_rate=growth_rate)
        x = TransitionLayer(x, reduction)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

model = DenseNet(input_shape=(224, 224, 3), depth=40, growth_rate=12, reduction=0.5, num_classes=8)

In [11]:
model.compile(optimizer='adam',
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])

In [12]:
epochs = 30
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=epochs)

Epoch 1/30


I0000 00:00:1731125511.938460    3485 service.cc:148] XLA service 0x7ff164095fc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731125511.938516    3485 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-11-08 23:11:52.302245: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731125513.660392    3485 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-08 23:12:22.929451: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng46{k2=0,k5=2,k14=4} for conv (f32[32,36,112,112]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,48,112,112]{3,2,1,0}, f32[36,48,1,1]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_s

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/home/viviane/apps/anaconda3/envs/pro1-cnn/lib/python3.12/runpy.py", line 198, in _run_module_as_main

  File "/home/viviane/apps/anaconda3/envs/pro1-cnn/lib/python3.12/runpy.py", line 88, in _run_code

  File "/home/viviane/apps/anaconda3/envs/pro1-cnn/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/viviane/apps/anaconda3/envs/pro1-cnn/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/viviane/apps/anaconda3/envs/pro1-cnn/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/viviane/apps/anaconda3/envs/pro1-cnn/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/viviane/apps/anaconda3/envs/pro1-cnn/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/home/viviane

# Evaluating

In [ ]:
history.history

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(val_ds, verbose=2)

In [ ]:
print(test_acc)

In [ ]:
predictions = model.predict(train_ds_test)
val_pred = np.argmax(predictions, axis=1)

In [ ]:
y_true = []
y_pred = []
for images, labels in val_ds_test:
  # Make predictions on the batch
  batch_pred = model.predict(images)
  y_pred.extend(list(batch_pred.argmax(axis=1)))  # Predicted labels
  y_true.extend(list(labels.numpy()))  # True labels from the batch

# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

In [ ]:
disp = ConfusionMatrixDisplay(cm, display_labels=classes_,)
disp.plot()

In [ ]:
report = classification_report(y_true, y_pred)
print(report)

# Save

In [ ]:
# model.save('models/breast_cancer_classification-densenet.h5')